In [1]:
import tqdm
import numpy as np
from sklearn.datasets import make_circles, make_blobs, make_moons
from sklearn.model_selection import train_test_split

In [3]:
X, y = make_circles(n_samples=1000, factor=0.1, noise=0.05, random_state=0)

In [5]:
import torch

In [6]:
data = torch.from_numpy(X).float()

In [7]:
K = 2
D = 2
N = len(X)
iters = 50

In [8]:
gamma = 10
dist = torch.cdist(data, data) ** 2
Kernel = torch.exp(-gamma * dist)

In [9]:
assign_idx = torch.randint(0, K, size=(N,))

In [10]:
def calculate_kernel_distance(data, Kernel, K, assign_idx):
    N, D = data.shape
    counts = torch.zeros(K)
    counts = torch.scatter_add(counts, 0, assign_idx, torch.ones(N)) + 1e-10
    clusterwise_dist = torch.zeros(N, K)
    clusterwise_dist = torch.scatter_add(clusterwise_dist, 1, assign_idx.repeat(N, 1), Kernel)
    mask = torch.zeros(N, K)
    mask = torch.scatter(mask, 1, assign_idx.unsqueeze(1), 1)
    incluster_dist = (mask * clusterwise_dist).sum(dim=0)
    distances = torch.diag(Kernel).unsqueeze(1) - 2 * clusterwise_dist / counts + incluster_dist / counts**2
    return distances

In [11]:
for i in tqdm.tqdm(range(iters)):
    distances = calculate_kernel_distance(data, Kernel, K, assign_idx)
    assign_idx = distances.argmin(dim=1)

100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 278.04it/s]
